In [1]:
#
# import utilitaires
#
import helpers as hlp
import helpers.visual_words as vw
import helpers.features as feat

In [2]:
config_overrides = hlp.MetaObject.from_json("config_overrides.json")

#
# features orb
#
orb_features_config = feat.OrbFeaturesConfig()
if not orb_features_config is None:
    hlp.MetaObject.override_from_object(orb_features_config, 
                                        config_overrides.orb.features)

orb_features = feat.load(orb_features_config, None)

print("ORB features", orb_features.features.shape)


#
# features sift
#
sift_features_config = feat.SiftFeaturesConfig()
if not sift_features_config is None:
    hlp.MetaObject.override_from_object(sift_features_config, 
                                        config_overrides.sift.features)

sift_features = feat.load(sift_features_config, None)

print("SIFT features", sift_features.features.shape)

ORB features (20908751, 32)
SIFT features (14386568, 128)


In [3]:
#
# parametres bag of visual words orb
#
orb_bovw_config = vw.BoVWConfig()
if not config_overrides is None:
    hlp.MetaObject.override_from_object(orb_bovw_config, 
                                        config_overrides.orb.bovw)

print("orb_bovw_config")
display(vars(orb_bovw_config))


# ATTENTION: cette etape peut etre longue a la 1e execution
#            elle est sans feedback visuel, soyez patient
with hlp.Profile() as orb_bovw_profile:
    orb_bovw = vw.load_bovw(orb_bovw_config, orb_features.features)

print("ORB BoVW loading:", orb_bovw_profile.round_duration(), "s")
print("ORB BoVW clusters:", orb_bovw.model.cluster_centers_.shape)

orb_bovw_config


{'install_path': 'dataset/OrbBoVW.pkl',
 'force_generate': False,
 'n_clusters': 500}

ORB BoVW loading: 0.03 s
ORB BoVW clusters: (500, 32)


In [4]:
#
# parametres bag of visual words sift
#
sift_bovw_config = vw.BoVWConfig()
if not config_overrides is None:
    hlp.MetaObject.override_from_object(sift_bovw_config, 
                                        config_overrides.sift.bovw)

print("sift_bovw_config")
display(vars(sift_bovw_config))

# ATTENTION: cette etape peut etre longue a la 1e execution
#            elle est sans feedback visuel, soyez patient
with hlp.Profile() as sift_bovw_profile:
    sift_bovw = vw.load_bovw(sift_bovw_config, sift_features.features)

print("SIFT BoVW loading:", sift_bovw_profile.round_duration(), "s")
print("SIFT BoVW clusters:", sift_bovw.model.cluster_centers_.shape)

sift_bovw_config


{'install_path': 'dataset/SiftBoVW.pkl',
 'force_generate': False,
 'n_clusters': 500}

SIFT BoVW loading: 0.06 s
SIFT BoVW clusters: (500, 128)
